### ChatOpenAI 사용하기
- 템플릿에 변수를 입력하여 prompt 생성
- ChatOpenAI 모델을 불러온 뒤, 모델에 prompt를 입력하여 결과 출력

In [8]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}", 

)
prompt = template.format(country_a="Metixco", country_b="Thailand")
chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)



'The distance between Metepec, Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [11]:

template = ChatPromptTemplate.from_messages(
    [
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your nanme?")
    ]
)

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand"
)

chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

### OutputParser로 결과 출력하기
- 출력 형태를 상황에 맞게 수정할 때 사용

In [15]:
from langchain.schema import BaseOutputParser

class CommanOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommanOutputParser()

p.parse("Hello,  how,are,you")

['Hello', 'how', 'are', 'you']

In [18]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. DO NOT replay with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommanOutputParser()
p.parse(result.content)

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

### Chain 사용하기
- Template, Chat model, Output Parser를 Chain으로 엮음
- chain.invoke()를 통해 Langchain에서 템플릿 내용을 모델에 입력한 뒤 결과를 parser에 넣어 출력함

In [19]:
# Chain of template, Language Model, and Output Parser
chain = template | chat | CommanOutputParser()

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']